<a href="https://colab.research.google.com/github/scaairesearch/da_bias_demo/blob/main/demo_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Demo - Bias

In [ ]:
# install general libraries
import os
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm # for beautiful model training updates

import matplotlib.pyplot as plt # for plots

import time
from pathlib import Path

from copy import deepcopy
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
import pandas as pd

### Loading CSV Files

In [ ]:
# checking the mounted drive and mounting if not done
if not os.path.exists('/content/gdrive'):
  from google.colab import drive
  drive.mount('/content/gdrive')
else:
    print("Google Drive is already mounted.")

Mounted at /content/gdrive


In [ ]:
# base_dir = ""
non_dann_result=pd.read_csv('/content/gdrive/MyDrive/CV_FER/results/Result dataframes/non_dann_result_23_05_wo_se_a.csv').sort_values(by=['image'])
dann_result=pd.read_csv('/content/gdrive/MyDrive/CV_FER/results/Result dataframes/dann_result_23_05_wo_se_a.csv').sort_values(by=['image'])
ewc_dann_result=pd.read_csv('/content/gdrive/MyDrive/CV_FER/results/Result dataframes/ewc_dann_result_23_05_wo_se_a.csv').sort_values(by=['image'])

In [ ]:
# Rename the last column in each dataframe
non_dann_result.rename(columns={'p_emotion': 'p_emotion_non_dann'}, inplace=True)
dann_result.rename(columns={'p_emotion': 'p_emotion_dann'}, inplace=True)
ewc_dann_result.rename(columns={'p_emotion': 'p_emotion_ewc_dann'}, inplace=True)

In [ ]:
non_dann_result

,Unnamed: 0,image,gt_emotion,gt_race,p_emotion_non_dann
167,167,afraid_African_214.jpg,Fear,African,Fear
2502,2502,afraid_american_238.jpg,Surprise,African,Fear
2069,2069,afraid_asian_103.jpg,Happy,Asian,Angry
1006,1006,afraid_asian_336.jpg,Surprise,Asian,Neutral
2424,2424,afraid_asian_367.jpg,Surprise,Asian,Neutral
...,...,...,...,...,...
4325,4325,unhappy_expression_537.jpg,Happy,White,Sad
2049,2049,unhappy_expression_643.jpg,Disgust,White,Disgust
8202,8202,unhappy_expression_730.jpg,Neutral,Asian,Disgust
8344,8344,unhappy_expression_749.jpg,Disgust,White,Happy


In [ ]:
merged_df = non_dann_result.merge(dann_result, on=['image', 'gt_emotion', 'gt_race']).merge(ewc_dann_result, on=['image', 'gt_emotion', 'gt_race'])


In [ ]:
merged_df.head()

,Unnamed: 0_x,image,gt_emotion,gt_race,p_emotion_non_dann,Unnamed: 0_y,p_emotion_dann,Unnamed: 0,p_emotion_ewc_dann
0,167,afraid_African_214.jpg,Fear,African,Fear,4064,Neutral,7694,Neutral
1,2502,afraid_american_238.jpg,Surprise,African,Fear,174,Neutral,293,Fear
2,2069,afraid_asian_103.jpg,Happy,Asian,Angry,4002,Angry,2763,Angry
3,1006,afraid_asian_336.jpg,Surprise,Asian,Neutral,845,Neutral,7041,Neutral
4,2424,afraid_asian_367.jpg,Surprise,Asian,Neutral,4416,Neutral,7260,Neutral


In [ ]:
merged_df.shape, merged_df.columns

((8458, 9),
 Index(['Unnamed: 0_x', 'image', 'gt_emotion', 'gt_race', 'p_emotion_non_dann',
        'Unnamed: 0_y', 'p_emotion_dann', 'Unnamed: 0', 'p_emotion_ewc_dann'],
       dtype='object'))

In [ ]:
merged_df = merged_df[['image', 'gt_emotion', 'gt_race', 'p_emotion_non_dann', 'p_emotion_dann', 'p_emotion_ewc_dann']]

In [ ]:
merged_df.shape, merged_df.columns

((8458, 6),
 Index(['image', 'gt_emotion', 'gt_race', 'p_emotion_non_dann',
        'p_emotion_dann', 'p_emotion_ewc_dann'],
       dtype='object'))

1. Images that are ok in non_dann, dann, dann_ewc
2. Images that are nok in non_dann, ok in dann and dann_ewc
3. Images that are nok in non_dann, nok in dann and ok in dann_ewc
4. Images that are nok in non_dann, dann_ewc but ok in dann
5. Images which are non_ok on all 3

### Condition 1: Images that are ok in non_dann, dann, dann_ewc

In [ ]:
condition_1 = (merged_df['gt_emotion'] == merged_df['p_emotion_non_dann']) & (merged_df['gt_emotion'] == merged_df['p_emotion_dann']) & ( merged_df['gt_emotion'] == merged_df['p_emotion_ewc_dann'])

In [ ]:
filtered_df_condition_1 = merged_df[condition_1]
filtered_df_condition_1.shape, filtered_df_condition_1.head()

((1805, 6),
                      image gt_emotion  gt_race p_emotion_non_dann  \
 5      afraid_asian_39.jpg    Neutral    Asian            Neutral   
 32  amazed_African_145.jpg      Happy  African              Happy   
 38  amazed_African_253.jpg      Happy  African              Happy   
 40   amazed_African_28.jpg      Happy  African              Happy   
 41  amazed_African_293.jpg    Neutral  African            Neutral   
 
    p_emotion_dann p_emotion_ewc_dann  
 5         Neutral            Neutral  
 32          Happy              Happy  
 38          Happy              Happy  
 40          Happy              Happy  
 41        Neutral            Neutral  )

In [ ]:
# df_merged_per_race = merged_df.groupby('gt_race').size().reset_index(name='merged_count')

In [ ]:
# filtered_df_condition_1_per_race= filtered_df_condition_1.groupby('gt_race').size().reset_index(name='condition_1_count')

In [ ]:
def race_proportion(base_df, condition_df, column_name = 'gt_race'):#, column_name):
  base_df_col = f'{column_name}'+'_base'
  condition_df_col = f'{column_name}'+'_condition'
  base_df_per_column_name = base_df.groupby(column_name).size().reset_index(name=base_df_col)
  condition_df_per_column_name = condition_df.groupby(column_name).size().reset_index(name=condition_df_col)
  merged_df = base_df_per_column_name.merge(condition_df_per_column_name, on=column_name)


  merged_df[condition_df_col] =   merged_df[condition_df_col].fillna(0)
  # # merged_df[merged_df.columns[-1]] =   merged_df[merged_df.columns[-1]].fillna(0)
  proportion_col = f'{column_name}'+'_proportion'
  merged_df[proportion_col] = round(merged_df[condition_df_col] *100/ merged_df[base_df_col],2)
  print(merged_df.head())

  return merged_df
  # merged_df[] = merged_df[column_name].fillna(0)
  # print(merged_df.head())

In [ ]:
race_proportion(merged_df, filtered_df_condition_1)

          gt_race  gt_race_base  gt_race_condition  gt_race_proportion
0         African           814                164               20.15
1           Asian          1072                212               19.78
2          Indian           580                102               17.59
3          Latino           194                 70               36.08
4  Middle Eastern           202                 33               16.34


,gt_race,gt_race_base,gt_race_condition,gt_race_proportion
0,African,814,164,20.15
1,Asian,1072,212,19.78
2,Indian,580,102,17.59
3,Latino,194,70,36.08
4,Middle Eastern,202,33,16.34
5,White,5596,1224,21.87


In [ ]:
race_proportion(df_merged_per_race,filtered_df_condition_1_per_race)

          gt_race  merged_count  condition_1_count  race_proportion
0         African           814                164            20.15
1           Asian          1072                212            19.78
2          Indian           580                102            17.59
3          Latino           194                 70            36.08
4  Middle Eastern           202                 33            16.34


### Condition 2: Images that are nok in non_dann, ok in dann and dann_ewc

In [ ]:
condition_2 = (merged_df['gt_emotion'] != merged_df['p_emotion_non_dann']) & (merged_df['gt_emotion'] == merged_df['p_emotion_dann']) & ( merged_df['gt_emotion'] == merged_df['p_emotion_ewc_dann'])

In [ ]:
filtered_df_condition_2 = merged_df[condition_2]
filtered_df_condition_2.shape, filtered_df_condition_2.head()

((278, 6),
                       image gt_emotion  gt_race p_emotion_non_dann  \
 18        afraid_man_41.jpg    Neutral  African            Disgust   
 31   amazed_African_141.jpg    Neutral  African            Disgust   
 242    amazed_black_160.jpg    Neutral  African                Sad   
 247    amazed_black_243.jpg    Neutral   Latino              Angry   
 267     amazed_boss_204.jpg      Angry    Asian              Happy   
 
     p_emotion_dann p_emotion_ewc_dann  
 18         Neutral            Neutral  
 31         Neutral            Neutral  
 242        Neutral            Neutral  
 247        Neutral            Neutral  
 267          Angry              Angry  )